In [33]:

import logging
logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger('bethmetamod')

import asyncio
import sys
from asyncio_extras.file import open_async

from pathlib import Path
import winreg
from utils import *
import psutil
import aiohttp
import shelve
from collections import namedtuple
from boltons.cacheutils import cachedproperty
from boltons.strutils import camel2under
from tqdm import tqdm_notebook as tqdm
import urllib.parse
import yaml
import re
from ntfsutils.hardlink import samefile, create as create_hardlink
import pefile

from datetime import datetime, timedelta
import subprocess


In [2]:
class Game:
    """Abstract base class representing a video game."""
    @classmethod
    def get_root_dir(cls, tries=5):
        try:
            return Path(get_regkey(r'HKLM', rf'SOFTWARE\WOW6432Node\Bethesda Softworks\{cls.REG_NAME}', 'installed path'))
        except FileNotFoundError:
            if tries:
                cls.ping_launcher()
                return cls.get_root_dir(tries=(tries - 1))
            else:
                raise

    @classmethod
    def ping_launcher(cls):
        """Start the Launcher and then immediately close it."""
        cls.start_steam()
        for i in range(60):
            print('waiting for 1 second')
            time.sleep(1)
            procs = map(psutil.Process, psutil.pids())
            ob_launcher, = (proc for proc in procs if proc.name() == cls.LAUNCHER_EXE)
            ob_launcher.kill()
    
    @classmethod
    def start_steam(cls):
        os.startfile(f'steam://run/{cls.STEAM_ID}')
        
    @cachedclassproperty
    def root_dir(cls):
        return cls.get_root_dir()
    
    @cachedclassproperty
    def game_exe(cls):
        return cls.root_dir / cls.GAME_EXE
    
    @cachedclassproperty
    def launcher_exe(cls):
        return cls.root_dir / cls.LAUNCHER_EXE
    
    @cachedclassproperty
    def tesxedit_exe(cls):
        return cls.root_dir / cls.TESXEDIT_EXE

class Oblivion(Game):
    REG_NAME = 'oblivion'
    STEAM_ID = '22330'
    LAUNCHER_EXE = 'OblivionLauncher.exe'
    GAME_EXE = 'Oblivion.exe'
    TESXEDIT_EXE = 'TES4Edit.exe'
    NEXUS_NAME = 'oblivion'
    

In [3]:
class Config:
    DOWNLOADS_DIR = Path(r'W:\bethmetamod-dls') #TODO
    DOWNLOADS_DB = Path(r'W:\bethmetamod-dls\downloads.shelve')
    MODS_DIR = Path(r'M:\bethmetamod\mods')
    LOGIN_PATH = Path(r'M:\bethmetamod\logins.yml')
    VANILLA_DIR = Path(r'M:\bethmetamod\vanilla')
    PURGED_DIR = Path(r'M:\bethmetamod\purged')
    CHUNK_SIZE = 1024
    
    
    @cachedclassproperty
    def game(cls):
        return Oblivion
    
    @cachedclassproperty
    def downloads_db(cls):
        return shelve.open(str(cls.DOWNLOADS_DB))
    
    @cachedclassproperty
    def login_info(cls):
        with cls.LOGIN_PATH.open('r') as f:
            return yaml.safe_load(f)
        

In [4]:
DownloadInfo = namedtuple('DownloadInfo', ('url', 'filename', 'size', 'sha256'))

In [5]:
class BaseDownload:
    async def extract(self, mod):
        dl_path = mod.dl_path / self.dl_info.filename
        await extract_path_async(dl_path, mod.mod_path)


In [6]:
async def extract_path_async(path, extractdir=None):
    _77PATH = r'C:\Program Files\7-Zip\7z.exe'
    
    extractdir = str(extractdir) or str(path.parent)
    #TODO add pbar?
    proc = await asyncio.create_subprocess_exec(
        str(_77PATH), 'x', '-y', str(path),
        cwd=extractdir,
    )
    stdout, stderr = await proc.communicate()
    if proc.returncode != 0:
        raise Exception(f'7zip returncode={proc.returncode}')

In [7]:
def recurse_files(path):
    """
    walks all files (not directories) in path and yields them one
    at a time (in arbitrary order) as paths relative to path
    """
    for root, dirs, files in os.walk(str(path)):
        root_path = Path(root)
        for file in files:
            yield (root_path / file).relative_to(path)

def recurse_all(path):
    for root, dirs, files in os.walk(str(path)):
        root_path = Path(root)
        for d in dirs:
            yield root_path / d
        for file in files:
            yield root_path / file


In [53]:
class NexusDownload(BaseDownload):
    cookies = None
    logged_in = None
    redirect_re = re.compile(r'(?ms).*?window\.location\.href = "(http://filedelivery\.nexusmods\.com[^"]*?)".*')
    
    @cachedclassproperty
    def login_url(cls):
        return f'https://www.nexusmods.com/{Config.game.NEXUS_NAME}/sessions/?Login'
    
    def __init__(self, nexus_id, game_name=None):
        self.nexus_id = nexus_id
        self.dl_info = None
        self.game_name = game_name or Config.game.NEXUS_NAME
    
    @classmethod
    async def login(cls, session):
        log.debug('NexusDownload.login called')
        if cls.logged_in is not None and (datetime.now() - cls.logged_in) < timedelta(hours=1):
            log.info('Already logged in.')
            session.cookie_jar.update_cookies(cls.cookies)
            return
        log.info('Logging into nexusmods.com')
        
        async with http_request(session, 'post', cls.login_url, data=Config.login_info['nexus']) as response:
            response.raise_for_status()

        cls.logged_in = datetime.now()
        cls.cookies = dict(session.cookie_jar)
        
    async def download(self, mod, session, force=False):
        try:
            self.dl_info = Config.downloads_db[f'{type(self).__name__}---{self.nexus_id}']
            dl_path = mod.dl_path / self.dl_info.filename
            if dl_path.exists() and sha256_path(dl_path) == self.dl_info.sha256:
                if force:                    
                    log.info(f'Forcing download that would not have happened except force=True')
                else:
                    log.info(f'{dl_path} already exists and passes hash check, skipping download')
                    return
        except KeyError:
            pass
        
        await self.login(session)
        url = f'http://www.nexusmods.com/{self.game_name}/ajax/downloadfile?id={self.nexus_id}&rrf'
        
        async with http_request(session, 'get', url) as response:
            response.raise_for_status()
            page_text = await response.text()
            new_url = self.redirect_re.match(page_text).group(1)
            
        async with http_request(session, 'get', new_url) as response:
            if self.dl_info:
                dl_filename = self.dl_info.filename
                dl_size = self.dl_info.size
            else:
                try:
                    dispo = response.headers['Content-Disposition']
                    _, dl_filename = dispo.split('filename=')
                    if dl_filename.startswith(('"',"'")):
                        dl_filename = localName[1:-1]
                except:
                    *_, dl_filename = urllib.parse.unquote(urllib.parse.urlparse(new_url).path).split('/')
                
                dl_size = response.headers.get('Context-Length')
                
            dl_path = mod.dl_path / dl_filename
  
            pbar = tqdm(total=dl_size)
            dl_path.parent.mkdir(parents=True, exist_ok=True)
            async with open_async(str(dl_path), 'wb') as f:
                async for chunk in chunked(response, Config.CHUNK_SIZE):
                    await f.write(chunk)
                    pbar.update(len(chunk))
            pbar.close()
            
            # TODO log.info
            Config.downloads_db[f'{type(self).__name__}---{self.nexus_id}'] = self.dl_info = DownloadInfo(
                url=new_url,
                filename=dl_filename,
                sha256=sha256_path(dl_path),
                size=dl_path.stat().st_size,
            )
           

In [9]:
# init vanilla folder
def init_vanilla():
    for path in recurse_files(Config.game.root_dir):
        r_path = Config.game.root_dir / path
        v_path = Config.VANILLA_DIR / path
        print(r_path, v_path)
        if not v_path.exists() or not samefile(str(r_path), str(v_path)):
            if v_path.exists():
                v_path.unlink()
            v_path.parent.mkdir(exist_ok=True, parents=True)
            create_hardlink(str(r_path), str(v_path))

In [10]:
class Download(BaseDownload):
    def __init__(self, url, **kwargs):
        self.url = url
        self.dl_info = None
        self.kwargs = kwargs
        
    async def download(self, mod, session, force=False):
        try:
            self.dl_info = Config.downloads_db[f'{type(self).__name__}---{self.url}']
            dl_path = mod.dl_path / self.dl_info.filename
            if dl_path.exists() and sha256_path(dl_path) == self.dl_info.sha256:
                if force:                    
                    log.info(f'Forcing download that would not have happened except force=True')
                else:
                    log.info(f'f{dl_path} already exists and passes hash check, skipping download')
                    return
        except KeyError:
            pass
        
        async with http_request(session, 'get', self.url, **self.kwargs) as response:          
            if self.dl_info:
                dl_filename = self.dl_info.filename
                dl_size = self.dl_info.size
            else:
                try:
                    dispo = response.headers['Content-Disposition']
                    _, dl_filename = dispo.split('filename=')
                    if dl_filename.startswith(('"',"'")):
                        dl_filename = localName[1:-1]
                except:
                    *_, dl_filename = urllib.parse.unquote(urllib.parse.urlparse(self.url).path).split('/')
                
                dl_size = response.headers.get('Content-Length')

            dl_path = mod.dl_path / dl_filename

            pbar = tqdm(total=dl_size)
            dl_path.parent.mkdir(parents=True, exist_ok=True)
            async with open_async(str(dl_path), 'wb') as f:
                async for chunk in chunked(response, Config.CHUNK_SIZE):
                    await f.write(chunk)
                    pbar.update(len(chunk))
            pbar.close()
            
            # TODO log.info
            Config.downloads_db[f'{type(self).__name__}---{self.url}'] = self.dl_info = DownloadInfo(
                url=self.url,
                filename=dl_filename,
                sha256=sha256_path(dl_path),
                size=dl_path.stat().st_size,
            )


In [15]:
class Mod:
    downloads = ()

    def __init__(self):
        downloads = []

    @cachedclassproperty
    def mod_name(cls):
        return camel2under(cls.__name__)    
    
    @cachedproperty
    def mod_path(self):
        return Config.MODS_DIR / self.mod_name
    
    @cachedproperty
    def dl_path(self):
        return Config.DOWNLOADS_DIR / self.mod_name
    
    async def download(self, session):
        for download in self.downloads:
            await download.download(self, session)
    
    async def preprocess(self):
        """extracting from archives, binary patching vanilla files."""
        pass
    
    async def extract(self, force=False):
        self.mod_path.mkdir(exist_ok=True, parents=True)
        if not force:
            try:
                next(self.mod_path.iterdir())
            except StopIteration:
                pass
            else:
                log.info(f'{self.mod_path} is not empty, extracted before, skipping extraction')
                return  # mod dir not empty
        for download in self.downloads:
            await download.extract(self)
        
    def modify(self):
        """
        Yield a tuple of (source path, dest_path) where source_path is an
        absolute path and dest_path is an path relative to game.root_dir
        """
        # find data dir
        candidates = set()
        for path in recurse_all(self.mod_path):
            if path.is_dir():
                if path.name.lower == 'data':
                    candidates.add(path)
                elif path.name.lower in ('textures', 'music', 'video', 'shaders', 'obse'):
                    candidates.add(path.parent)
            elif path.name.lower().endswith(('.bsa', '.esp', '.esm')):
                candidates.add(path.parent)
        
        assert len(candidates) == 1
        
        root_path, = candidates
        
        for path in recurse_files(root_path):
            yield root_path / path, Path('./data') / path
        
    
    async def postprocess(self):
        """Edit ini files, I/O unintensive stuff that must be performed after modify."""
        pass

In [16]:
class OBSE(Mod):
    downloads = [
        Download('http://obse.silverlock.org/download/obse_0021.zip'),
        Download('http://obse.silverlock.org/download/obse_loader.zip'),
    ]
    def modify(self):
        for path in recurse_files(self.mod_path):
            if path.parts[0] == 'src':
                continue
            yield self.mod_path / path, path

class FarCryGrass(Mod):
    downloads = [
        NexusDownload('1000014269')
    ]

class OBSETester(Mod):
    # requires OBSE
    downloads = [
        NexusDownload('65277')
    ]

class OneTweak(Mod):
    downloads = [
        NexusDownload('1000231728', game_name='skyrim')
    ]
    def modify(self):
        for path in self.mod_path.glob(r'*/SKSE/plugins/*'):
            yield path, Path('./Data/OBSE/plugins') / path.name
        

In [51]:
class ENB(Mod):
    downloads = [
        Download('http://enbdev.com/enbseries_oblivion_v0181.zip',
                 headers={'referer': 'http://enbdev.com/mod_tesoblivion_v0181.htm'})
    ]
    
    def modify(self):
        for path in self.mod_path.glob('WrapperVersion/*'):
            yield path, Path('.') / path.name

class ENBoost(Mod):
    # requires ENB
    downloads = [
        NexusDownload('1000007218')
    ]
    def __init__(self, gpu=None, os=None):
        #TODO autodetect
        if gpu is None:
            lines = subprocess.check_output('wmic path win32_videocontroller get /format:list').decode('utf8').splitlines()
            pairs = (s.split('=', 1) for s in lines if '=' in s)
            adapters = {v for k, v in pairs if k == 'AdapterCompatibility'}

            nvidia_present = 'NVIDIA' in adapters
            intel_present = 'Intel' in adapters
            amd_present = 'AMD' in adapters

            assert nvidia_present or amd_present
            if nvidia_present:
                gpu = 'NVidia'
            elif amd_present:
                gpu = 'AMD'
            else:
                assert False
            
            log.info(f'autodetected gpu: {gpu}')

        if os is None:
            import platform
            os = f'{platform.architecture()[0]}OS'
            log.info(f'autodetected os architecture: {os}')

        assert gpu in {'NVidia', 'AMD'}
        assert os in {'64bitOS', '32bitOS'}
        self.gpu = gpu
        self.os = os
    
    def modify(self):
        yield (self.mod_path / self.gpu / self.os / 'enblocal.ini'), Path('enblocal.ini')
        
class FourGBPatch(Mod):
    async def preprocess(self):
        old_path = Config.VANILLA_DIR / Config.game.GAME_EXE
        new_path = self.mod_path / Config.game.GAME_EXE
        if new_path.exists():
            return
        pe = pefile.PE(str(old_path))
        pe.FILE_HEADER.IMAGE_FILE_LARGE_ADDRESS_AWARE = True
        pe.write(filename=str(new_path))
        pe.close()
        del pe
    
    def modify(self):
        yield self.mod_path / Config.game.GAME_EXE, Path('.') / Config.game.GAME_EXE

In [23]:
class MoreHeap(Mod):
    downloads = [
        NexusDownload('1000006402')
    ]
    def modify(self):
        yield self.mod_path / 'Version.dll', Path('Version.dll')

In [52]:
if sys.platform == 'win32':
    loop = asyncio.ProactorEventLoop()
    asyncio.set_event_loop(loop)
else:
    loop = asyncio.get_event_loop()
    
async def main(loop):
    mod_list = [
        FourGBPatch(),
        OBSE(),
        OneTweak(),
        OBSETester(),
        ENB(),
        ENBoost(),
        MoreHeap(),
    ]
    converged_paths = {}
    for path in recurse_files(Config.VANILLA_DIR):
        converged_paths[str(path).lower()] = Config.VANILLA_DIR / path
        
    for mod in mod_list:
        async with aiohttp.ClientSession(loop=loop) as session:  
            await mod.download(session)
            
    if False:  #stop after download?
        return
            
    for mod in mod_list:
        await mod.extract()
        
    if False:  #stop after extract?
        return
    
    for mod in mod_list:
        await mod.preprocess()
    
    for mod in mod_list:
        for source_path, dest_path in mod.modify():
            converged_paths[str(dest_path).lower()] = source_path
        
    for dest_path, source_path in converged_paths.items():
        dest_path = Config.game.root_dir / dest_path
        if not dest_path.exists() or not samefile(str(dest_path), str(source_path)):
            if dest_path.exists():
                dest_path.unlink()  # FIXME move to purged dir?
            dest_path.parent.mkdir(exist_ok=True, parents=True)
            create_hardlink(str(source_path), str(dest_path))
    
    for path in recurse_files(Config.game.root_dir):
        if (
            str(path).lower() not in converged_paths and 
            not str(path).endswith(('.ini', '.cfg', '.xml', '.json'))
        ): 
            purged_path = Config.PURGED_DIR / datetime.now().isoformat().replace(':', '') / path
            purged_path.parent.mkdir(exist_ok=True, parents=True)
            (Config.game.root_dir / path).rename(purged_path)
            
    #TODO purge empty directories somehow?
            
    for mod in mod_list:
        await mod.postprocess()

loop.run_until_complete(main(loop))

DEBUG:asyncio:Using proactor: IocpProactor
INFO:bethmetamod:autodetected gpu: NVidia
INFO:bethmetamod:autodetected os architecture: 64bitOS
INFO:bethmetamod:fW:\bethmetamod-dls\obse\obse_0021.zip already exists and passes hash check, skipping download
INFO:bethmetamod:fW:\bethmetamod-dls\obse\obse_loader.zip already exists and passes hash check, skipping download
INFO:bethmetamod:fW:\bethmetamod-dls\one_tweak\OneTweak-40706-2-1-0-2.7z already exists and passes hash check, skipping download
INFO:bethmetamod:fW:\bethmetamod-dls\obse_tester\OBSE Test Plugin-33574.rar already exists and passes hash check, skipping download
INFO:bethmetamod:fW:\bethmetamod-dls\enb\enbseries_oblivion_v0181.zip already exists and passes hash check, skipping download
INFO:bethmetamod:fW:\bethmetamod-dls\en_boost\ENBoost 1_0-45266-259.zip already exists and passes hash check, skipping download
INFO:bethmetamod:fW:\bethmetamod-dls\more_heap\MoreHeap 11-44985-1-1.zip already exists and passes hash check, skipping